In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
import schedulers

import tensorflow as tf
assert tf.config.list_physical_devices('GPU')

In [2]:
BATCH_SIZE = 32

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)

def build_model(UNITS=380):  # EMBED_DIM=28
    inp = keras.Input(batch_shape=(None, None), batch_size=BATCH_SIZE)
    layer = layers.Embedding(LETTERS_SIZE, UNITS, mask_zero=True)(inp)
    
    bidi = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')
    layer = bidi(layer)
    layer = layers.add([layer, bidi(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=inp, outputs=outputs)

    return model

model = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 380)    16720       input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, None, 380)    2313440     embedding[0][0]                  
                                                                 bidirectional[0][0]              
__________________________________________________________________________________________________
add (Add)                       (None, None, 380)    0           bidirectional[0][0]          

In [4]:
# masked version of accuracy and sce
def accuracy(real, pred):
    acc = tf.keras.metrics.sparse_categorical_accuracy(real, pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(real, 0)), dtype=acc.dtype)
    acc *= mask

    return tf.reduce_sum(acc) / tf.reduce_sum(mask)

def sparse_categorical_crossentropy(y_true, y_pred, sample_weight=None):
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(y_true, 0)), dtype=loss.dtype)
    loss *= mask

    return tf.reduce_sum(loss) / tf.reduce_sum(mask)

def fit(train_validation, scheduler=None, verbose=1, lr=1e-4, epochs=1):
    train, valid = train_validation
    model.compile(loss=masked_sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics=[masked_accuracy])
    callbacks = []
    if isinstance(scheduler, schedulers.CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin }
    
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=epochs, verbose=verbose, callbacks=callbacks)


MAXLEN = 64
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [5]:
data_mix = load_data(['poetry', 'rabanit', 'pre_modern'])

In [21]:
model.load_weights('./checkpoints/uninit')
history = fit(data_mix, scheduler=schedulers.CircularLearningRate(30e-4, 80e-4, 1e-4))
model.save_weights('./checkpoints/mix')

4367/4367 [==============================] - 247s 57ms/step - loss: 0.2993 - N_loss: 0.2325 - D_loss: 0.0591 - S_loss: 0.0078 - N_masked_accuracy: 0.9187 - D_masked_accuracy: 0.9772 - S_masked_accuracy: 0.9980 - val_loss: 0.2672 - val_N_loss: 0.1966 - val_D_loss: 0.0645 - val_S_loss: 0.0062 - val_N_masked_accuracy: 0.9351 - val_D_masked_accuracy: 0.9754 - val_S_masked_accuracy: 0.9984


In [12]:
data_modern = load_data(validation=0.2, source=['modern'])

In [14]:
model.load_weights('./checkpoints/mix')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(50e-4, 60e-4, 1e-5))  #  EMBED_DIM=28, UNITS=253: val_N_accuracy: 0.9575 - val_D_accuracy: 0.9856 - val_S_accuracy: 0.9994
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(1e-4, 6e-4, 1e-5), epochs=2)
model.save_weights('./checkpoints/modern')

469/469 [==============================] - 30s 64ms/step - loss: 0.1691 - N_loss: 0.1266 - D_loss: 0.0397 - S_loss: 0.0028 - N_masked_accuracy: 0.9573 - D_masked_accuracy: 0.9851 - S_masked_accuracy: 0.9994 - val_loss: 0.1665 - val_N_loss: 0.1258 - val_D_loss: 0.0387 - val_S_loss: 0.0020 - val_N_masked_accuracy: 0.9580 - val_D_masked_accuracy: 0.9852 - val_S_masked_accuracy: 0.9995


In [15]:
model.load_weights('./checkpoints/modern')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(1e-4, 6e-4, 1e-5), epochs=3)
model.save_weights('./checkpoints/modern_over')

Epoch 1/3
469/469 [==============================] - 34s 73ms/step - loss: 0.0698 - N_loss: 0.0508 - D_loss: 0.0180 - S_loss: 0.0010 - N_masked_accuracy: 0.9836 - D_masked_accuracy: 0.9936 - S_masked_accuracy: 0.9998 - val_loss: 0.1590 - val_N_loss: 0.1194 - val_D_loss: 0.0378 - val_S_loss: 0.0018 - val_N_masked_accuracy: 0.9610 - val_D_masked_accuracy: 0.9860 - val_S_masked_accuracy: 0.9995
Epoch 2/3
469/469 [==============================] - 27s 57ms/step - loss: 0.0526 - N_loss: 0.0379 - D_loss: 0.0139 - S_loss: 8.0444e-04 - N_masked_accuracy: 0.9881 - D_masked_accuracy: 0.9951 - S_masked_accuracy: 0.9998 - val_loss: 0.1604 - val_N_loss: 0.1201 - val_D_loss: 0.0385 - val_S_loss: 0.0018 - val_N_masked_accuracy: 0.9616 - val_D_masked_accuracy: 0.9859 - val_S_masked_accuracy: 0.9995
Epoch 3/3
469/469 [==============================] - 27s 58ms/step - loss: 0.0417 - N_loss: 0.0298 - D_loss: 0.0112 - S_loss: 6.4986e-04 - N_masked_accuracy: 0.9909 - D_masked_accuracy: 0.9961 - S_masked_ac

In [23]:
model.load_weights('./checkpoints/modern')
model.save('modern.h5')

In [24]:
import tensorflowjs as tfjs
# model.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(model, '.')

C:\Users\elaza\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflowjs\converters\keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [8]:
model.load_weights('./checkpoints/modern_over')

def print_predictions(data, s):
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(data.text[s], ts=batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(data.text[s], ts=batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    total = []
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        last = expected_niqqud[i].tolist().index(0)
        res = expected_niqqud[i][:last] == actual_niqqud[i][:last]
        total.extend(res)
        print(round(np.mean(res), 2), f'({last - sum(res)} out of {last})')
        print()
    print(round(np.mean(total), 3))

print_predictions(data_modern[1], slice(0, None))

מצוי:  הַלִּיקוֹפֶּטֶר. כָּל כְּלֵי הָרֶכֵּב עָבְרוּ הַמָּרָה מְפוֹרֶטֶת לִשְׂדֵה הַקְּרָב וְכָל אֶחָד בָּנוּי 
רצוי:  הֵלִיקוֹפְּטֵר. כָּל כְּלֵי הָרֶכֶב עָבְרוּ הֲמָרָה מְפוֹרֶטֶת לִשְׂדֵה הַקְּרָב וְכָל אֶחָד בָּנוּי 
0.92 (5 out of 62)

מצוי:  הַתַּחְתּוֹנִים". כָּכָה, הָא? קְהִילַּת הַמּוֹדִים בָּאִינְטֶרְנֵט - לְטִיפּוּלְכֶם... 
רצוי:  הַתַּחְתּוֹנִים". כָּכָה, הָא? קְהִילַּת הַמּוֹדִים בָּאִינְטֶרְנֵט - לְטִיפּוּלְכֶם... 
1.0 (0 out of 56)

מצוי:  יַנְחוּ אֶת הַשּׁוֹטְרִים לָגֶשֶׁת לְאֶזְרָחִים וּלְבַקֵּשׁ מֵהֶם לְעֲטּוֹת מַסֵכָה, מֵאַחַר שֶׁבִּידֵי 
רצוי:  יַנְחוּ אֶת הַשּׁוֹטְרִים לָגֶשֶׁת לָאֶזְרָחִים וּלְבַקֵּשׁ מֵהֶם לַעֲטוֹת מַסֵּכָה, מֵאַחַר שֶׁבִּידֵי 
0.97 (2 out of 62)

מצוי:  וְסוּרְיָה. הָאֲכִיפָה שֶׁל אוֹתָם פִּסְקֵי דִּין נַעֲשֵׂית בְּרַחֲבֵי הָעוֹלָם נֶגֶד נְכָסִים 
רצוי:  וְסוּרְיָה. הָאֲכִיפָה שֶׁל אוֹתָם פִּסְקֵי דִּין נַעֲשֵׂית בְּרַחֲבֵי הָעוֹלָם נֶגֶד נְכָסִים 
1.0 (0 out of 60)

מצוי:  בְּאִישׁוּן לֵיל טֵרוֹרִיסְט שֶׁנִּמְצָא בַּבִּנְיָין. שָׁעָה שֶׁאַתָה ע

In [ ]:
for i in range(50):
    model.load_weights('./checkpoints/pre_modern')
    p1 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    p2 = np.exp(np.random.uniform(low=np.log(1e-4), high=np.log(1e-1)))
    p3 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    print(p1, p2, p3, end=', ', sep=', ')
    history = fit(data_modern, scheduler=schedulers.CircularLearningRate(p1, p2, p3), verbose=0)
    print(history.history['val_N_accuracy'][0])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [6]:
import hebrew
print(hebrew.ENDINGS_TO_REGULAR)

{'ך': 'כ', 'ם': 'מ', 'ן': 'נ', 'ף': 'פ', 'ץ': 'צ'}
